Начальные условия

* M - множество состояний
* К - множество возможных значений марковского процесса(выбросы). Выбросы - это e, за которыми следуют целые числа от 0 до k-1.
* Путь состояния закодирован аналогичным образом с префиксом p



Задание 1 


<font size="3"> Считать матрицу  вероятностей перехода между состояниями (MxM) и матрицу условных вероятностей (P($e_i$ | $_j$)
где $e_i$ - наблюдение, $_j$ - состояние, и начальное распределение (вектор длины M) из файла (файл имеет формат с разделенной табуляцией)</font>


Для того, чтобы считать матрицу вероятностей, попросим пользователя ввести некоторые параметры, а именно M, K и введем параметр length, который предполагает длину последовательности

In [1]:
import numpy as np
import pandas as pd

In [2]:
M = int(input('Введите мощность множества состояний: '))

Введите мощность множества состояний: 5


In [4]:
K=int(input('Введите мощность множества возможных значений марковского процесса: '))

Введите мощность множества возможных значений марковского процесса: 6


In [5]:
length = int(input('Введите длину последовательности: '))

Введите длину последовательности: 7


Далее генерируем параметры модели.

In [46]:
transition_matrix = pd.DataFrame(np.random.randint(0,10,size=(M, M))).add_prefix('p_').T.add_prefix('p_').T
emission_matrix = pd.DataFrame(np.random.randint(0,10,size=(M, K))).add_prefix('e_').T.add_prefix('p_').T
pi = pd.DataFrame(np.random.randint(1,10,size=(M, 1))).T.add_prefix('p_').T

transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1),axis=0)
emission_matrix = emission_matrix.div(emission_matrix.sum(axis=1),axis=0)
pi = pi.div(pi.sum(axis=0),axis=1)


Запишем в csv файл параметры модели.

In [47]:
transition_matrix.to_csv('transition_matrix.csv',sep='\t')
emission_matrix.to_csv('emission_matrix.csv',sep='\t')
pi.to_csv('pi.csv',sep='\t')

Прочитаем csv файл.

In [48]:
transition_matrix_read = pd.read_csv('transition_matrix.csv',sep='\t',index_col=0)
emission_matrix_read = pd.read_csv('emission_matrix.csv',sep='\t',index_col=0)
pi_read = pd.read_csv('pi.csv',sep='\t',index_col=0)

In [49]:
transition_matrix_read

,p_0,p_1,p_2,p_3,p_4
p_0,0.142857,0.238095,0.285714,0.238095,0.095238
p_1,0.176471,0.411765,0.117647,0.176471,0.117647
p_2,0.238095,0.047619,0.190476,0.095238,0.428571
p_3,0.300000,0.100000,0.166667,0.300000,0.133333
p_4,0.000000,0.310345,0.275862,0.103448,0.310345


In [50]:
emission_matrix_read

,e_0,e_1,e_2,e_3,e_4,e_5
p_0,0.181818,0.409091,0.181818,0.136364,0.000000,0.090909
p_1,0.142857,0.142857,0.095238,0.142857,0.285714,0.190476
p_2,0.058824,0.117647,0.176471,0.264706,0.176471,0.205882
p_3,0.281250,0.218750,0.093750,0.062500,0.281250,0.062500
p_4,0.173913,0.173913,0.043478,0.043478,0.304348,0.260870


In [51]:
pi_read

,0
p_0,0.115385
p_1,0.346154
p_2,0.346154
p_3,0.076923
p_4,0.115385


Выбираем состояние для генерации пути

In [52]:
def random_choice(distribution,p):
    s = np.random.random()*distribution[p].sum()
    for i in range(len(distribution)):
        s = s - distribution[p]['p_'+str(i)]
        if s < 0:
            return 'p_'+str(i)

Выбираем матрицу условных вероятностей

In [53]:
   def random_choice2(distribution,p):
    s = np.random.random()*distribution[p].sum()
    for i in range(len(distribution)):
        s = s - distribution[p]['e_'+str(i)]
        if s < 0:
            return 'e_'+str(i)



Создали функцию для генерации пути

In [54]:
def HMM_random_generator_path(begin_distribution1,transition1,L):
    path = []
    path.append(random_choice(begin_distribution1,'0'))
    for i in range(1,L):
        path.append(random_choice(pd.DataFrame(transition1.T[path[i-1]]),path[i-1]))
    return path

Создали функцию для генераци последовательности

In [55]:
def HMM_random_generator_sequence(states,emission1,L):
    sequence = []
    for i in range(length):
        sequence.append(random_choice2(pd.DataFrame(emission1.T[states[i]]),states[i]))
    return sequence

Генерируем путь и последовательность

In [56]:
simulated_path = HMM_random_generator_path(pi_read,transition_matrix_read,length)
sequence = HMM_random_generator_sequence(simulated_path,emission_matrix_read,length)

In [57]:
simulated_path

['p_2', 'p_3', 'p_2', 'p_0', 'p_1', 'p_1', 'p_4']

In [58]:
sequence

['e_2', 'e_1', 'e_2', 'e_2', 'e_4', 'e_2', 'e_4']

Считать параметры скрытой марковской модели и наблюдаемую последовательность X. Используйте алгоритм Витерби для нахождения наиболее вероятного пути , которая дает последовательность X и параметры марковской модели.
На выходе: путь в виде файла с форматом с разделенной табуляцией.


Долнительная функция для получения максимального элемента в сумме вероятностей

In [59]:
def get_max(values):
    maxv = values[0]
    maxi = 0
    for ind, val in enumerate(values):
        if val>maxv:
            maxv = val
            maxi = ind
    return maxv,maxi

Ветерби алгоритм

In [60]:
def viterbi(transition_matrix_read,emission_matrix_read,sequence1):
    F = pd.DataFrame(np.zeros((M,length+1)),index=transition_matrix_read.index)
    F[0]=pi_read['0']
    C = pd.DataFrame(np.zeros((M,length+1)),index=transition_matrix_read.index)
    for k in F.columns[1:]:
        for i in F.index:
            values = []
            for j in F.index:
                values.append(F[k-1][j]*transition_matrix_read[i][j]*emission_matrix_read[sequence1[k-1]][i])
            maxv, maxi = get_max(values)
            F[k][i] = maxv
            C[k][i] = maxi
    return F, C



In [61]:
F, C= viterbi(transition_matrix_read,emission_matrix_read,sequence)

In [62]:
F

,0,1,2,3,4,5,6,7
p_0,0.115385,0.014985,0.001133,0.000043,2.473689e-06,0.000000e+00,9.035389e-09,0.000000e+00
p_1,0.346154,0.013575,0.000799,0.000031,1.228004e-06,1.682781e-07,8.274989e-09,9.735281e-10
p_2,0.346154,0.011635,0.000504,0.000057,2.146440e-06,1.247238e-07,1.362939e-08,4.581309e-10
p_3,0.076923,0.005727,0.000780,0.000025,9.502470e-07,1.656488e-07,4.658872e-09,6.050484e-10
p_4,0.115385,0.006450,0.000867,0.000012,1.064762e-06,2.799705e-07,3.777712e-09,1.777747e-09


In [63]:
C

,0,1,2,3,4,5,6,7
p_0,0.0,2.0,2.0,3.0,2.0,0.0,3.0,0.0
p_1,0.0,1.0,1.0,1.0,1.0,0.0,4.0,1.0
p_2,0.0,2.0,0.0,0.0,0.0,0.0,4.0,2.0
p_3,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0
p_4,0.0,2.0,2.0,4.0,2.0,2.0,4.0,2.0


Найти вероятность P(x), используя прямой алгоритм (хранить полную матрицу вероятностей, найденных при прямом алгоритме  fk(xi), для того чтобы использовать его для последующего декодирования). 


In [64]:
def forward(pi_read,transition_matrix_read,emission_matrix_read,sequence):
    F = np.zeros((length,M))
    F[0] = pi_read['0']*emission_matrix_read[sequence[0]]
    for i in range(1,length):
        F[i] = F[i-1]@transition_matrix_read*emission_matrix_read[sequence[i]]
    return pd.DataFrame(F).add_prefix('p_').T

In [65]:
def P(data,length):
    return data[lenqth-1].sum()


In [66]:
F = forward(pi_read,transition_matrix_read,emission_matrix_read,sequence)


In [67]:
F

,0,1,2,3,4,5,6
p_0,0.020979,0.010441,0.000733,0.000113,0.000000,2.265607e-06,0.000000e+00
p_1,0.032967,0.003394,0.000601,0.000056,0.000020,1.775290e-06,5.056835e-07
p_2,0.061086,0.002835,0.001109,0.000112,0.000014,2.653891e-06,3.156632e-07
p_3,0.007212,0.004225,0.000492,0.000052,0.000019,1.227223e-06,4.379313e-07
p_4,0.005017,0.006013,0.000219,0.000033,0.000025,8.067218e-07,6.013923e-07


Запрограммировать обратный алгоритм.

In [76]:
def backward(F,transition_matrix_read,emission_matrix_read,sequence1):
    F1 = pd.DataFrame(np.zeros((M,length)),index=transition_matrix_read.index)
    F1[len(sequence)-1]=F[len(sequence)-1]
    for j in range(len(sequence)-2,-1,-1):
        for i in F1.index:
            p_sum = 0
            for k in F1.index:
                p_sum += F1[j+1][k]*transition_matrix_read[k][i]*emission_matrix_read[sequence1[j+1]][k]
            F1[j][i] = p_sum
    return F1

In [77]:
F1=backward(F,transition_matrix_read,emission_matrix_read,sequence)

In [78]:
F1

,0,1,2,3,4,5,6
p_0,6.725923e-12,3.627336e-11,3.044615e-10,2.543209e-09,1.286512e-08,9.707345e-08,0.000000e+00
p_1,7.031253e-12,3.311482e-11,2.849785e-10,2.639390e-09,1.170810e-08,1.093145e-07,5.056835e-07
p_2,7.351040e-12,3.123867e-11,2.758431e-10,2.325505e-09,1.149443e-08,1.076634e-07,3.156632e-07
p_3,8.664277e-12,3.765464e-11,3.112393e-10,2.188080e-09,1.264783e-08,8.508710e-08,4.379313e-07
p_4,4.860667e-12,2.828437e-11,2.530457e-10,3.009238e-09,1.104815e-08,1.297506e-07,6.013923e-07
